## Imports

In [9]:
# DO NOT CHANGE THESE LINES.
import os
import pandas as pd
import json
import warnings
from joblib import load
warnings.filterwarnings('ignore')

## Paths

In [10]:
# DO NOT CHANGE THESE LINES.
ROOT_DIR = os.path.dirname(os.getcwd())
MODEL_INPUTS_OUTPUTS = os.path.join(ROOT_DIR, 'model_inputs_outputs/')
INPUT_DIR = os.path.join(MODEL_INPUTS_OUTPUTS, "inputs")
OUTPUT_DIR = os.path.join(MODEL_INPUTS_OUTPUTS, "outputs")
INPUT_SCHEMA_DIR = os.path.join(INPUT_DIR, "schema")
DATA_DIR = os.path.join(INPUT_DIR, "data")
TRAIN_DIR = os.path.join(DATA_DIR, "training")
TEST_DIR = os.path.join(DATA_DIR, "testing")
MODEL_PATH = os.path.join(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = os.path.join(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'ohe.joblib')
LABEL_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'label_encoder.joblib')
PREDICTOR_DIR_PATH = os.path.join(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = os.path.join(PREDICTOR_DIR_PATH, "predictor.joblib")
IMPUTATION_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'imputation.joblib')
PREDICTIONS_DIR = os.path.join(OUTPUT_DIR, 'predictions')
PREDICTIONS_FILE = os.path.join(PREDICTIONS_DIR, 'predictions.csv')
if not os.path.exists(PREDICTIONS_DIR):
    os.makedirs(PREDICTIONS_DIR)

### Reading the schema

In [11]:
file_name = [f for f in os.listdir(INPUT_SCHEMA_DIR) if f.endswith('.json')][0]
schema_path = os.path.join(INPUT_SCHEMA_DIR, file_name)
with open(schema_path, "r", encoding="utf-8") as file:
    schmea = json.load(file)
features = schmea['features']

numeric_features = []
categorical_features = []
for f in features:
    if f['dataType'] == 'CATEGORICAL':
        categorical_features.append(f['name'])
    else:
        numeric_features.append(f['name'])

id_feature = schmea['id']['name']
target_feature = schmea['target']['name']

### Reading test data.

In [12]:
file_name = [f for f in os.listdir(TEST_DIR) if f.endswith('.csv')][0]
file_path = os.path.join(TEST_DIR, file_name)
df = pd.read_csv(file_path)
df.head()

,u_id,fatals,a_ct,a_ped_f,a_pedal_f,a_roll,a_hr,a_polpur,month,day,...,a_body,owner,deaths,numoccs,impact1,deformed,ve_forms,ve_total,weather,lgt_cond
0,41633,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,10,7,...,"Motorcycles, Mopeds, All-Terrain Vehicles; All...",Driver (in this crash) Was Registered Owner,1,1.0,NaN,Minor damage,1,3,Clear,Dark - not lighted
1,38966,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,6,9,...,Automobiles,Driver (in this crash) Was Registered Owner,1,1.0,NaN,Disabling damage,1,1,Clear,Dark - lighted
2,24020,1,Single-Vehicle Crash,Pedestrian Fatality Involved Crash,Other Crash,Other Crash,Yes - Hit and Run,Other Crash,10,15,...,Van-Based Light Trucks,Driver (in this crash) Not Registered Owner (o...,0,1.0,Clockpoint 1,Minor damage,1,1,Clear,Dark - not lighted
3,52280,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,2,14,...,"Motorcycles, Mopeds, All-Terrain Vehicles; All...",Driver (in this crash) Not Registered Owner (o...,1,1.0,NaN,Disabling damage,1,1,Clear,Dark - not lighted
4,47480,1,Single-Vehicle Crash,Other Crash,Pedalcyclist Fatality Involved Crash,Other Crash,No - Hit and Run,Other Crash,5,2,...,Utility Vehicles,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 12,NaN,1,1,Clear,Dark - lighted


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [13]:
columns_with_missing_values = df.columns[df.isna().any()]
imputaion_values = load(IMPUTATION_FILE)
for column in columns_with_missing_values:
    df[column].fillna(imputaion_values[column], inplace=True)


In [14]:
df

,u_id,fatals,a_ct,a_ped_f,a_pedal_f,a_roll,a_hr,a_polpur,month,day,...,a_body,owner,deaths,numoccs,impact1,deformed,ve_forms,ve_total,weather,lgt_cond
0,41633,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,10,7,...,"Motorcycles, Mopeds, All-Terrain Vehicles; All...",Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 12,Minor damage,1,3,Clear,Dark - not lighted
1,38966,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,6,9,...,Automobiles,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Dark - lighted
2,24020,1,Single-Vehicle Crash,Pedestrian Fatality Involved Crash,Other Crash,Other Crash,Yes - Hit and Run,Other Crash,10,15,...,Van-Based Light Trucks,Driver (in this crash) Not Registered Owner (o...,0,1.0,Clockpoint 1,Minor damage,1,1,Clear,Dark - not lighted
3,52280,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,2,14,...,"Motorcycles, Mopeds, All-Terrain Vehicles; All...",Driver (in this crash) Not Registered Owner (o...,1,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Dark - not lighted
4,47480,1,Single-Vehicle Crash,Other Crash,Pedalcyclist Fatality Involved Crash,Other Crash,No - Hit and Run,Other Crash,5,2,...,Utility Vehicles,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Dark - lighted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11317,45653,1,Single-Vehicle Crash,Pedestrian Fatality Involved Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,8,19,...,Automobiles,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Daylight
11318,46252,1,Single-Vehicle Crash,Other Crash,Other Crash,Rollover Involved Crash,No - Hit and Run,Other Crash,5,26,...,Automobiles,Driver (in this crash) Was Registered Owner,1,2.0,Clockpoint 1,Disabling damage,1,1,Clear,Dawn
11319,38231,1,Single-Vehicle Crash,Pedestrian Fatality Involved Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,11,5,...,Utility Vehicles,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 1,Functional damage,1,1,Clear,Dark - lighted
11320,6491,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,7,2,...,Van-Based Light Trucks,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 12,Disabling damage,1,2,Clear,Daylight


### Encoding
We encode the data using the same encoder that we saved during training.

In [15]:
# Saving the id column in a different variable.
ids = df[id_feature]

# Dropping the id from the dataframe
df.drop(columns=[id_feature], inplace=True)

# Encoding the rest of the features
encoder = load(OHE_ENCODER_FILE)
encoder.fit(df)
df = encoder.transform(df)


### Making predictions
Using the model saved during training. Notice that the model outputs a 2D array with many rows and 3 columns. </br>
Each row in the array represents an answer to a sample in the test data. Each number of the 3 numbers in the row is a probability to one of the 3 classes in the original problem.

In [18]:
model = load(PREDICTOR_FILE_PATH)

l1 = list(df.columns)
l2 = model.feature_names_in_

not_common = [x for x in l2 if x not in l1]

print(not_common)
predictions = model.predict_proba(df)

predictions

Index(['fatals', 'month', 'day', 'hour', 'minute', 'age', 'permvit',
       'pernotmvit', 'mod_year', 'deaths',
       ...
       'weather_Rain', 'weather_Fog, smog, smoke', 'weather_Snow',
       'weather_Reported as unknown', 'lgt_cond_Daylight',
       'lgt_cond_Dark - not lighted', 'lgt_cond_Dark - lighted',
       'lgt_cond_Dawn', 'lgt_cond_Dark - unknown lighting', 'lgt_cond_Dusk'],
      dtype='object', length=102)
['fatals' 'month' 'day' 'hour' 'minute' 'age' 'permvit' 'pernotmvit'
 'mod_year' 'deaths' 'numoccs' 've_forms' 've_total'
 'a_ct_Single-Vehicle Crash' 'a_ped_f_Other Crash'
 'a_ped_f_Pedestrian Fatality Involved Crash' 'a_pedal_f_Other Crash'
 'a_pedal_f_Pedalcyclist Fatality Involved Crash' 'a_roll_Other Crash'
 'a_roll_Rollover Involved Crash' 'a_hr_No - Hit and Run'
 'a_hr_Yes - Hit and Run' 'a_polpur_Other Crash'
 'a_polpur_Police Pursuit Involved Crash' 'day_week_sat' 'day_week_sun'
 'day_week_fri' 'day_week_thu' 'day_week_mon' 'day_week_tue'
 'a_dow_type_Weekday

array([[0.25837236, 0.58910519, 0.15252245],
       [0.46769766, 0.38508777, 0.14721457],
       [0.01852313, 0.96843804, 0.01303883],
       ...,
       [0.05025896, 0.90245294, 0.04728811],
       [0.10367076, 0.81904393, 0.07728531],
       [0.39320075, 0.39234509, 0.21445416]])

### Getting the original labels.
To get the original labels back, we use the same encoder from the training phase. <br><br>
Instead of calling the transform() function, this time we use inverse_transform().<br><br>

This will convert the labels [ 1, 2, 3 ] to the original labels [ drunk_driver_involved, other,	speeding_driver_involved ] 

In [17]:
encoder = load(LABEL_ENCODER_FILE)

class_names = encoder.inverse_transform([0, 1, 2])

predictions = pd.DataFrame(predictions, columns=class_names)
predictions.insert(0, 'u_id', ids)
predictions.to_csv(PREDICTIONS_FILE)
predictions


,u_id,drunk_driver_involved,other,speeding_driver_involved
0,41633,0.258372,0.589105,0.152522
1,38966,0.467698,0.385088,0.147215
2,24020,0.018523,0.968438,0.013039
3,52280,0.394031,0.427425,0.178544
4,47480,0.205888,0.598923,0.195189
...,...,...,...,...
11317,45653,0.048489,0.912331,0.039180
11318,46252,0.448500,0.250659,0.300841
11319,38231,0.050259,0.902453,0.047288
11320,6491,0.103671,0.819044,0.077285
